# Generate a SLIM forcing file from CTSM output
by S. Levis, 
modified from pres_vs_hist_alb_LN_postAGU20190621_corrected_rs.ipynb by Marysa Lague

In [73]:
import os
import sys

import time as tm
from copy import copy 

import netCDF4 as nc
import xarray as xr
import numpy as np

## Accurate masks

In [74]:
# USER MODIFY section
# -------------------
# CTSM history file from archived simulation that used the fsurdat file a few lines down
# simulation must be a bgc case, so as to include the history variable HTOP
casename = 'ihist_bgccrop'  # USER MODIFY
start_yr = '2014'  # USER MODIFY

username = os.environ.get('USER')
case_archive_dir = '/glade/scratch/' + username + '/archive/' + casename  # USER may need to modify
ctsm_dir = case_archive_dir + '/lnd/hist/'  # USER may need to modify
ctsm_file = ctsm_dir + casename + '.clm2.h0.' + start_yr + '-01.nc'  # USER may need to modify

if os.path.exists(ctsm_file):
    ctsm_ds = xr.open_dataset(ctsm_file)
else:
    errmsg = "ctsm_file does not exist: " + ctsm_file
    sys.exit(errmsg)

lat_ctsm = ctsm_ds.variables['lat'].values[:]
lon_ctsm = ctsm_ds.variables['lon'].values[:]
landmask = ctsm_ds.landmask.values[:]
dims = np.shape(landmask)

# USER MODIFY section
# -------------------
# ctsm fsurdat file from simulation that produced the ctsm history file a few lines up
# if ctsm fsurdat file does not exist, glc_mask will equal zero everywhere
surfdat_dir ='/glade/p/cesmdata/cseg/inputdata/lnd/clm2/surfdata_map/release-clm5.0.18/'  # USER may need to modify
surfdat_file = surfdat_dir + 'surfdata_0.9x1.25_hist_78pfts_CMIP6_simyr2000_c190214.nc'  # USER may wish to modify

glc_mask = np.zeros(dims)
if os.path.exists(surfdat_file):
    surfdat_ds = xr.open_dataset(surfdat_file)
    # get glacier mask
    glc_pct = (surfdat_ds.variables['PCT_GLACIER']).values[:]
    # apply the glacier mask where glc_pct > 50% 
    glc_mask[glc_pct > 50] = 1
else:
    surfdata_file = ''

dirt_mask = np.where((landmask==1.) & (glc_mask==0.), 1.0, 0.0)

In [75]:
# USER-GENERATED FILES
# Load nco-generated ctsm history file containing monthly averages (ie, avg Jan, avg Feb, etc)
# Sample use of nco to concatenate 12 months of a single year:
# ncecat <casename>.clm2.h0.<year>-* <casename>.clm2.h0.<year>.nc
# nco user's guide: https://nco.sourceforge.net/
ctsm_concatenated_file = ctsm_dir + casename + '.clm2.h0.' + start_yr + '.nc'  # USER may need to modify

if os.path.exists(ctsm_concatenated_file):
    ds = xr.open_dataset(ctsm_concatenated_file, decode_times=False)
else:
    errmsg = "ctsm_concatenated_file does not exist: " + ctsm_concatenated_file
    sys.exit(errmsg)

ds['time'] = ds['record']  # ncecat (a few lines up) introduced the "record" dimension
months_per_yr = len(ds['time'].values)
print(months_per_yr)  # expect 12

# DUST: nco-generated monthly means of coupler history files
# if dust_file does not exist, dust fluxes will be set to zero
cpl_dir = case_archive_dir + '/cpl/hist/'  # USER may need to modify
dust_file = cpl_dir + casename + '.cpl.h0.' + start_yr + '.nc'  # USER may need to modify

12


In [76]:
# ctsm history variables used in this script; for informational purposes only
# lndvars = ['FSR','FSDS','GSSHA','GSSUN','GSSHALN','GSSUNLN','TLAI','HTOP',
#            'LAISUN','LAISHA','SNOW_DEPTH',
#            'FSRND','FSRNI','FSRVD','FSRVI',
#            'FSDSND','FSDSNI','FSDSVD','FSDSVI' ]
# add  mask information to the dataset
ds['glc_mask'] = xr.DataArray(dims=['lat','lon'], data=glc_mask)
ds['bareground_mask'] = xr.DataArray(dims=['lat','lon'], data=dirt_mask)

In [77]:
# Construct albedos from sw reflected / down:
#-------ALBEDO------#
nameswap = {}
nameswap['FSR'] = 'ALBEDO'
# albedo:
ds_temp = xr.merge([ds['FSR'], ds['FSDS']])
ds_temp['ALBEDO'] = ds_temp['FSR'] / ds_temp['FSDS']
ds_temp['ALBEDO'].attrs['units'] = 'unitless'
ds_temp['ALBEDO'].attrs['longname'] = 'multistream albedo'
ds['ALBEDO'] = ds_temp['ALBEDO']

ds_temp = xr.merge([ds['FSRND'], ds['FSDSND']])
ds_temp['ALBEDO_ND'] = ds_temp['FSRND'] / ds_temp['FSDSND']
ds_temp['ALBEDO_ND'].attrs['units'] = 'unitless'
ds_temp['ALBEDO_ND'].attrs['longname'] = 'near-IR direct albedo'
ds['ALBEDO_ND'] = ds_temp['ALBEDO_ND']

ds_temp = xr.merge([ds['FSRNI'], ds['FSDSNI']])
ds_temp['ALBEDO_NI'] = ds_temp['FSRNI'] / ds_temp['FSDSNI']
ds_temp['ALBEDO_NI'].attrs['units'] = 'unitless'
ds_temp['ALBEDO_NI'].attrs['longname'] = 'near-IR diffuse albedo'
ds['ALBEDO_NI'] = ds_temp['ALBEDO_NI']
    
ds_temp = xr.merge([ds['FSRVD'], ds['FSDSVD']])
ds_temp['ALBEDO_VD'] = ds_temp['FSRVD'] / ds_temp['FSDSVD']
ds_temp['ALBEDO_VD'].attrs['units'] = 'unitless'
ds_temp['ALBEDO_VD'].attrs['longname'] = 'visible direct albedo'
ds['ALBEDO_VD'] = ds_temp['ALBEDO_VD']

ds_temp = xr.merge([ds['FSRVI'], ds['FSDSVI']])
ds_temp['ALBEDO_VI'] = ds_temp['FSRVI'] / ds_temp['FSDSVI']
ds_temp['ALBEDO_VI'].attrs['units'] = 'unitless'
ds_temp['ALBEDO_VI'].attrs['longname'] = 'visible diffuse albedo'
ds['ALBEDO_VI'] = ds_temp['ALBEDO_VI']
    
#-------EVAP RS------#
gs_to_rs = 42.3 * 10**6  # umol H20/m2/s   to   s/m
ds_temp = xr.merge([ds['GSSUNLN'], ds['GSSHALN'], ds['LAISUN'], ds['LAISHA']])
sunLN = ds['GSSUNLN'] * ds['LAISUN']
shaLN = ds['GSSHALN'] * ds['LAISHA']
ds_temp['evap_rs_LN'] = gs_to_rs / (sunLN + shaLN)
ds_temp['evap_rs_LN'].attrs['units'] = 's/m'
ds_temp['evap_rs_LN'].attrs['longname'] = 'evaporative resistance at local noon = (42.3 x 10^6)/(gssunln*laisun + gsshaln*laisha)'
ds['evap_rs_LN'] = ds_temp['evap_rs_LN']
    
rs_LN_uncapped = gs_to_rs / (sunLN + shaLN)
rs_LN_capped = rs_LN_uncapped.copy()
rs_LN_capped.values = np.where(rs_LN_uncapped > 1000., 1000., rs_LN_uncapped)
ds_temp['evap_rs_LN_capped'] = rs_LN_capped
ds_temp['evap_rs_LN_capped'].attrs['units'] = 's/m'
ds_temp['evap_rs_LN_capped'].attrs['longname'] = 'evaporative resistance at local noon capped at 1000 s/m; else = (42.3 x 10^6)/(gssunln*laisun + gsshaln*laisha)'
ds['evap_rs_LN_capped'] = ds_temp['evap_rs_LN_capped']
    
del ds_temp

In [78]:
# Snow albedos calculated by Marysa L elsewhere
s_alb = {}
s_alb['vd'] = 0.97333038
s_alb['vi'] = 0.965662
s_alb['nd'] = 0.66046935
s_alb['ni'] = 0.7067166

## Get JJA snowmask

In [79]:
snow_thresh = 0.01  # m
snow = ds['SNOW_DEPTH'][6:9,:,:]  # JJA
jja_snowfree = np.where(snow > snow_thresh , np.nan , 1.0).mean()

## Calculate the values for albedo, rs, and hc:

In [80]:
alb = {}

alb['vd'] = ds['ALBEDO_VD']
alb['vi'] = ds['ALBEDO_VI']
alb['nd'] = ds['ALBEDO_ND']
alb['ni'] = ds['ALBEDO_NI']

# make a new albedo field using values anywhere there wasn't snow
nc_alb = {}
nc_alb['ground'] = {}
nc_alb['snow'] = {}

# do this in np arrays, not datasets
alb_ocn = 0.1  # set ocean points to this generic value
for a in alb.keys():
    nc_alb['ground'][a] = np.where(jja_snowfree==1., alb[a], alb[a])
    
    # put snow albedos where glacier mask is true
    nc_alb['ground'][a] = np.where(glc_mask==1., s_alb[a], nc_alb['ground'][a])
    
    # get rid of nans on ocean points
    nc_alb['ground'][a] = np.where(np.isnan(nc_alb['ground'][a]), alb_ocn, nc_alb['ground'][a]).mean(axis=0)
    
    # snow albedo just a single block of color:
    nc_alb['snow'][a] = np.ones(np.shape(landmask)) * s_alb[a]
    nc_alb['snow'][a] = np.where(np.isnan(nc_alb['snow'][a]), alb_ocn, nc_alb['snow'][a])

rs values:

In [81]:
nc_rs = {}
nc_rs = ds['evap_rs_LN_capped']

hc values:

In [82]:
# From BATS: glacier roughness 0.01  - constant
glc_hc = 0.01

# doesn't varry seasonally... I don't think... but do the seasonal mod anyhow. For consistency. 
nc_hc = {}
nc_hc = ds['HTOP']

# no nans: set to 0.01 (very smooth). 
hmin=0.01
nc_hc = np.where(np.isnan(nc_hc), hmin , nc_hc)

# Also no zeros, messes up the turbulence calculation. Make those smooth, too.
nc_hc = np.where(nc_hc < hmin, hmin, nc_hc)

# set glacier "height" to 0.01
nc_hc = np.where(glc_mask==1., glc_hc, nc_hc)


Other required values:

In [83]:
if os.path.exists(dust_file):
    dust_ds = xr.open_dataset(dust_file)
    dust1 = (dust_ds.variables['l2xavg_Fall_flxdst1']).values
    dust2 = (dust_ds.variables['l2xavg_Fall_flxdst2']).values
    dust3 = (dust_ds.variables['l2xavg_Fall_flxdst3']).values
    dust4 = (dust_ds.variables['l2xavg_Fall_flxdst4']).values
    # clobber nans
    dust1 = np.where(np.isnan(dust1), 0.0, dust1)
    dust2 = np.where(np.isnan(dust2), 0.0, dust2)
    dust3 = np.where(np.isnan(dust3), 0.0, dust3)
    dust4 = np.where(np.isnan(dust4), 0.0, dust4)
else:
    dust1 = 0
    dust2 = 0
    dust3 = 0
    dust4 = 0
    dust_file = ''

In [84]:
soil_cv_str = '2e6'
soil_cv_val = 2.0e6  # [J/m3/K]
soil_tk_val = 1.5  # W/m/K
glc_cv = 1.9e6  #  [J/m3/K]
glc_tk = 2.4  #  W/m/K
snow_mask_depth = 50.0 # kg/m2
bucket_capacity = 200.0 # kg/m2

# dummy array of ones to extend a lat x lon array into time: mon x lat x lon
stretch = np.ones([months_per_yr, dims[0], dims[1]])

In [85]:
# set up a dictionary called nc_data that stores all the variables to be output
nc_data = {}    # empty dictionary

# glacier mask:
nc_data['glc_mask'] = np.copy(glc_mask[:] * stretch)

# Albedos as alb_[g-ground/s-snow][v-visible/n-nir][d-direct/f-diffuse]
nc_data['alb_gvd'] = nc_alb['ground']['vd'] * stretch
nc_data['alb_svd'] = nc_alb['snow']['vd'] * stretch
nc_data['alb_gnd'] = nc_alb['ground']['nd'] * stretch
nc_data['alb_snd'] = nc_alb['snow']['nd'] * stretch
nc_data['alb_gvf'] = nc_alb['ground']['vi'] * stretch
nc_data['alb_svf'] = nc_alb['snow']['vi'] * stretch
nc_data['alb_gnf'] = nc_alb['ground']['ni'] * stretch
nc_data['alb_snf'] = nc_alb['snow']['ni'] * stretch

# Bucket capacity in [kg/m2] (or equivalently [mm])
nc_data['bucketdepth'] = np.copy(bucket_capacity * stretch)

# snow masking "depth" in water mass equivalent ([kg/m2] or [mm])
nc_data['snowmask'] = np.copy(snow_mask_depth * stretch)

# Emissivity (1 = perfect blackbody, not physically realistic)
nc_data['emissivity'] = np.copy(1.0 * stretch)

# Roughness as "vegetation height" [m] (which is then scaled down in the model 
#       as .1*veg height for actual roughness used)
nc_data['roughness'] = nc_hc.mean(axis=0) * stretch

# evaporative resistance [s/m] as a sort of "bulk stomatal resistance" - actual
#       resistance is calculated as a combination of this and how full the bucket is
# Initial pass, set all roughness to 100 (this is our "base" for glaciers also)
nc_data['evap_res'] = nc_rs.mean(axis=0) * stretch

# Dust fluxes (from clm4.5 coupled run). There are 4 different dust bins, each
#       is given its own field here, to avoid problems I ran into trying to read
#       netcdf fields with depth dimensions in the actual model code
nc_data['l2xavg_Fall_flxdst1'] = np.copy(dust1 * stretch)
nc_data['l2xavg_Fall_flxdst2'] = np.copy(dust2 * stretch)
nc_data['l2xavg_Fall_flxdst3'] = np.copy(dust3 * stretch)
nc_data['l2xavg_Fall_flxdst4'] = np.copy(dust4 * stretch)

# Soil Type (not currently used, set to 0)
nc_data['soil_type'] = np.copy(0.0 * stretch)

# Thermal Properties
#   soil heat capacity cv [J/m3/K] (uniform across column using this definition)
# ranges: 1.5e6 for gravel to 3 for clay/silt; 4.2 for water (if we go very saturated, but the dirt'll still be in there...)
nc_data['soil_cv_1d'] = np.copy(soil_cv_val * stretch)

#   soil thermal conductivity tk [W/m/K] (uniform across column using this definition)
nc_data['soil_tk_1d'] = np.copy(soil_tk_val * stretch)

#   ice (glacier) heat capacity cv [J/m3/K] (uniform across column using this definition)
# cv water = 4.188e6 , cv ice = 1.9415e+06     
#     near -20 C
nc_data['glc_cv_1d'] = np.copy(glc_cv * stretch)

#   ice (glacier) thermal conductivity tk [W/m/K] (uniform across column using this definition)
#     near -20 C
nc_data['glc_tk_1d'] = np.copy(glc_tk * stretch)

## Put all the vars into an xarray dataset

In [86]:
# Define a time vector for months 1-12
time_vect = range(months_per_yr + 1)[1:months_per_yr + 1]

attrs = {'Units'}

ds_slim = {}

# Write out to the new file:
ds_slim = xr.Dataset({'glc_mask': (['time','lsmlat','lsmlon'], nc_data['glc_mask']),
                      'alb_gvd': (['time','lsmlat','lsmlon'], nc_data['alb_gvd']),
                      'alb_svd': (['time','lsmlat','lsmlon'], nc_data['alb_svd']),
                      'alb_gnd': (['time','lsmlat','lsmlon'], nc_data['alb_gnd']),
                      'alb_snd': (['time','lsmlat','lsmlon'], nc_data['alb_snd']),
                      'alb_gvf': (['time','lsmlat','lsmlon'], nc_data['alb_gvf']),
                      'alb_svf': (['time','lsmlat','lsmlon'], nc_data['alb_svf']),
                      'alb_gnf': (['time','lsmlat','lsmlon'], nc_data['alb_gnf']),
                      'alb_snf': (['time','lsmlat','lsmlon'], nc_data['alb_snf']),
                      'bucketdepth': (['time','lsmlat','lsmlon'], nc_data['bucketdepth']),
                      'emissivity': (['time','lsmlat','lsmlon'], nc_data['emissivity']),
                      'snowmask': (['time','lsmlat','lsmlon'], nc_data['snowmask']), 
                      'roughness': (['time','lsmlat','lsmlon'], nc_data['roughness']),
                      'evap_res': (['time','lsmlat','lsmlon'], nc_data['evap_res']),
                      'l2xavg_Fall_flxdst1': (['time','lsmlat','lsmlon'], nc_data['l2xavg_Fall_flxdst1']),
                      'l2xavg_Fall_flxdst2': (['time','lsmlat','lsmlon'], nc_data['l2xavg_Fall_flxdst2']),
                      'l2xavg_Fall_flxdst3': (['time','lsmlat','lsmlon'], nc_data['l2xavg_Fall_flxdst3']),
                      'l2xavg_Fall_flxdst4': (['time','lsmlat','lsmlon'], nc_data['l2xavg_Fall_flxdst4']),
                      'soil_type': (['time','lsmlat','lsmlon'], nc_data['soil_type']),
                      'soil_tk_1d': (['time','lsmlat','lsmlon'], nc_data['soil_tk_1d']),
                      'soil_cv_1d': (['time','lsmlat','lsmlon'], nc_data['soil_cv_1d']),
                      'glc_tk_1d': (['time','lsmlat','lsmlon'], nc_data['glc_tk_1d']),
                      'glc_cv_1d': (['time','lsmlat','lsmlon'], nc_data['glc_cv_1d'])},
                      coords = {'lsmlon': (['lsmlon'], lon_ctsm),
                                'lsmlat': (['lsmlat'], lat_ctsm), 
                                'time': (['time'], time_vect)},
                      attrs = {'Author': username,
                               'Date_created': tm.strftime("%Y-%m-%d %H:%M:%S") + ' ' + tm.tzname[0] + ' ' + tm.tzname[1],
                               'Resolution': 'see surfdat_file listed below',
                               'Description': 'SLIM surdat file',
                               'ccesm_source_run': casename,
                               'ctsm_file': ctsm_file,
                               'dust_file': dust_file,
                               'surfdat_file_for_glc_mask': surfdat_file,
                              }
                    )

# Define units of each variable, and map them onto the dataset
# TODO ...something similar for other variable attributes, eg. valid_range, etc.
# For guidance: https://cfconventions.org/Data/cf-conventions/cf-conventions-1.10/cf-conventions.html#attribute-appendix
attr_map = {'glc_mask': ['unitless', 9.99e20], 
            'alb_gvd': ['unitless', 9.99e20], 
            'alb_svd': ['unitless', 9.99e20],
            'alb_gnd': ['unitless', 9.99e20], 
            'alb_snd': ['unitless', 9.99e20], 
            'alb_gvf': ['unitless', 9.99e20],
            'alb_svf': ['unitless', 9.99e20], 
            'alb_gnf': ['unitless', 9.99e20], 
            'alb_snf': ['unitless', 9.99e20],
            'bucketdepth': ['kg/m2', 9.99e20], 
            'emissivity': ['unitless', 9.99e20], 
            'snowmask': ['kg/m2', 9.99e20],
            'roughness': ['m', 9.99e20], 
            'evap_res': ['s/m', 9.99e20],
            'l2xavg_Fall_flxdst1': ['unknown', 9.99e20], 
            'l2xavg_Fall_flxdst2': ['unknown', 9.99e20], 
            'l2xavg_Fall_flxdst3': ['unknown', 9.99e20], 
            'l2xavg_Fall_flxdst4': ['unknown', 9.99e20], 
            'soil_type': ['unknown', 9.99e20], 
            'soil_tk_1d': ['W/m/K', 9.99e20], 
            'soil_cv_1d': ['J/m3/K', 9.99e20], 
            'glc_tk_1d': ['W/m/K', 9.99e20], 
            'glc_cv_1d': ['J/m3/K', 9.99e20], 
            'lsmlat': ['degrees north', False],
            'lsmlon': ['degrees east', False], 
            'time': ['month', False]}

for var, val in attr_map.items():
    ds_slim[var].attrs['Units'] = val[0]
    ds_slim[var].attrs['_FillValue'] = val[1]

## Write out the new datasets

In [87]:
ds_slim.to_netcdf(path = 'surdat_' + tm.strftime("%Y%m%d") + '.nc', format = 'NETCDF3_64BIT')